In [2]:
from transformers import GPT2LMHeadModel


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
n_embed = 384
n_head = 4 

head_dim = n_embed // n_head
head_dim

96

In [2]:
 
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


--2025-04-21 14:41:34--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... failed: No route to host.
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... failed: No route to host.
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... ^C


In [3]:
import requests

def download_shakespeare_dataset():
    url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        
        # Save the content to a file
        with open("input.txt", "w", encoding="utf-8") as f:
            f.write(response.text)
        
        print("Dataset downloaded successfully!")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading the dataset: {e}")

download_shakespeare_dataset()

Dataset downloaded successfully!


In [4]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()
data = text[:1000] #first 1000 characters
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [5]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode(data)
print(tokens[:24])

[5962, 22307, 25, 198, 8421, 356, 5120, 597, 2252, 11, 3285, 502, 2740, 13, 198, 198, 3237, 25, 198, 5248, 461, 11, 2740, 13]


In [16]:
import torch
buf = torch.tensor(tokens[:24+1])
print(buf.shape)
x = buf[:-1].view(4, 6)
y = buf[1:].view(4, 6)
print(x.shape, '\n',  x)
print(y.shape, '\n',  y)







torch.Size([25])
torch.Size([4, 6]) 
 tensor([[ 5962, 22307,    25,   198,  8421,   356],
        [ 5120,   597,  2252,    11,  3285,   502],
        [ 2740,    13,   198,   198,  3237,    25],
        [  198,  5248,   461,    11,  2740,    13]])
torch.Size([4, 6]) 
 tensor([[22307,    25,   198,  8421,   356,  5120],
        [  597,  2252,    11,  3285,   502,  2740],
        [   13,   198,   198,  3237,    25,   198],
        [ 5248,   461,    11,  2740,    13,   198]])


In [ ]:
# standard deviation grows inside the residual stream 
x = torch.zeros(768)
n = 100 # e.g. 100 layers
for i in range(n):
    # keeps std 1
    x += n**-0.5 * torch.randn(768)

print(x.std())


tensor(1.0070)


In [24]:
import torch
# set seed 
torch.random.manual_seed(42)

# simple mlp
net = torch.nn.Sequential(
    torch.nn.Linear(16, 32),
    torch.nn.GELU(),
    torch.nn.Linear(32, 1)
)

x = torch.randn(4, 16)
y = torch.randn(4, 1)
net.zero_grad()
yhat = net(x)
loss = torch.nn.functional.mse_loss(yhat, y)
loss.backward()
print(net[0].weight.grad.view(-1)[:10])

tensor([-0.0556, -0.0415,  0.0208,  0.0922, -0.0034, -0.1156,  0.1279,  0.0024,
         0.1107, -0.0587])
